In [1]:
%load_ext sql
import sqlite3

In [2]:
%sql sqlite:///CourseDataWarehouse.db

'Connected: @CourseDataWarehouse.db'

# 8. Demonstration Questions

## A. Do we need to increase the number of Intro to Accounting Courses offered due to the increase in student enrollment?

In [27]:
%%sql
SELECT CATALOG_ID, TERM, REMAINING
FROM COURSE_FACTS
WHERE CATALOG_ID = 'AC 0011'and TERM = 'Fall2018'
ORDER BY REMAINING DESC

 * sqlite:///CourseDataWarehouse.db
Done.


CATALOG_ID,TERM,REMAINING
AC 0011,Fall2018,2
AC 0011,Fall2018,2
AC 0011,Fall2018,2
AC 0011,Fall2018,2
AC 0011,Fall2018,2
AC 0011,Fall2018,2
AC 0011,Fall2018,1
AC 0011,Fall2018,1
AC 0011,Fall2018,1
AC 0011,Fall2018,1
